In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import pandas as pd
import os
from __future__ import division
import numpy as np
import cPickle as pickle
import matplotlib.pyplot as plt
from scipy import stats
import loadmat

In [3]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta/'

## calculate single trial values

In [1]:
import ShadePlots_elecs_stats as SP

In [2]:
SP.shadeplots_elecs_stats()

ValueError: too many values to unpack

## PCA outlier rejection
* all electrodes have the same number of trials
* each feature can have a different number of trials
* save separate file for each feature, plus an RT file for each feature to match number of trials dropped
* for unsmoothed data
* edited for surrogates

In [6]:
filename = os.path.join(SJdir,'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df_all = pd.read_csv(filename)
std_thresh = 4

In [7]:
#outlier rejection of RTs to match outlier rejection of PCA for each param
features_to_consider = ['means']

subjs, tasks, num_total, num_to_drop, feature_list = [[] for i in range(5)]

for s_t in df_all.groupby(['subj','task']):
        
    subj, task = s_t[0]

    #find outlier trials
    for f in features_to_consider:
        filename = os.path.join(SJdir, 'PCA', 'ShadePlots_hclust', 'elecs', 'significance_windows', 'unsmoothed', 'csv_files', '_'.join([subj, task, f]) + '.csv')
        df = pd.read_csv(filename)

        filename = os.path.join(SJdir, 'PCA', 'ShadePlots_hclust', 'elecs', 'significance_windows','unsmoothed', 'csv_files', '_'.join([subj, task, 'RTs']) + '.csv')
        df_RT = pd.read_csv(filename)

        trials_to_drop = df.apply(lambda x: ((x > (x.mean() + x.std(ddof = 1)*std_thresh)) | (x < (x.mean() - x.std(ddof = 1)* std_thresh))))

        outliers = np.where(trials_to_drop)[0]

        #drop outliers
        df.iloc[outliers] = np.nan
        df = df.dropna()

        df_RT.iloc[outliers] = np.nan
        df_RT = df_RT.dropna()

        filename = os.path.join(SJdir, 'PCA', 'Stats', 'outliers','for_PCA', 'unsmoothed', '_'.join([subj, task, f + '.csv']))
        df.to_csv(filename, index = False)
        filename = os.path.join(SJdir, 'PCA', 'Stats', 'outliers','for_PCA', 'unsmoothed', '_'.join([subj, task, f + '_RTs.csv'])) #RTs per elec (identical) with correct number of trials
        df_RT.to_csv(filename, index = False)

        subjs.append(subj)
        tasks.append(task)
        num_total.append(trials_to_drop.shape[0])
        num_to_drop.append(len(outliers))
        feature_list.append(f)


In [8]:
#make a summary
df_tally = pd.DataFrame({'subj':subjs, 'task': tasks, 'total' : num_total, 'outliers' : num_to_drop, 'feature':feature_list})
df_tally['percent_dropped'] = df_tally['outliers']/df_tally['total'] * 100
df_tally = df_tally[['subj','task','feature', 'outliers','total','percent_dropped']]

In [9]:
filename = os.path.join(SJdir, 'PCA', 'Stats', 'outliers', 'for_PCA','unsmoothed', 'tally_all.csv')
df_tally.to_csv(filename)

## combine all of df_thresh into 1 csv
* thresholded PC loading matrices (from PCA_thresh.py)


In [12]:
#params = ['means', 'medians', 'maxes', 'maxes_rel', 'stds']
params = ['means']

filename = os.path.join(SJdir,'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df_pattern = pd.read_csv(os.path.join(filename))

subj_task = df_pattern[['subj', 'task']].drop_duplicates()

for param in params:
    df_all = pd.DataFrame({'subj':[],'task':[], 'elec':[], 'pattern':[], 'ROI':[], 'max_pc':[], 'pc_list':[]})

    for s_t in subj_task.itertuples():
        _, subj, task = s_t
        filename = os.path.join(SJdir, 'PCA','Stats','Networks','zscore','PCA_'+ param,'networks', '_'.join([subj, task,'thresh.csv']))
        if os.path.isfile(filename):
            df = pd.read_csv(filename)
            df= df[['subj','task','elec','pattern','ROI','max_pc','pc_list']]
            df_all = df_all.append(df)
    df_all = df_all[['subj','task','elec','ROI','pattern','max_pc','pc_list']]
    filename = os.path.join(SJdir, 'PCA','Stats', 'Networks', 'zscore', 'PCA_'+ param, 'networks', 'all_subjects_thresh_'+ param +'.csv')
    df_all.to_csv(filename)

In [13]:
filename

'/home/knight/matar/MATLAB/DATA/Avgusta/PCA/Stats/Networks/zscore/PCA_means/networks/all_subjects_thresh_means.csv'

## threshold correlation matrices based on threshholded PC1 
* (or PC2)
* use PCA_threhsold.py using loading matrices from PYTHON/PCA_R/PCA_elecs.R

In [8]:
filename = os.path.join(SJdir,'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df = pd.read_csv(os.path.join(filename))

subj_task = df[['subj', 'task']].drop_duplicates()

param = 'means'
pc = 'PC1'

for s_t in subj_task.itertuples():
    _, subj, task = s_t

    filename = os.path.join(SJdir, 'PCA','Stats', 'Networks', 'zscore', 'PCA_'+param, 'networks', '_'.join([subj, task, 'thresh.csv']))
    df_thresh = pd.read_csv(filename)

    filename = os.path.join(SJdir, 'PCA','Stats','outliers', 'for_PCA', 'zscore', '_'.join([subj, task, param+'.csv']))
    df_data = pd.read_csv(filename)
    df_corr = df_data.corr()
    
    #skip PC2 for subj/task that only have PC1
    if not(pc in df_thresh.columns):
        continue
        
    #find elecs that belong to PC group
    idx = ~np.isnan(df_thresh[pc])
    a = df_thresh[idx].elec
    
    #index into correlation matrix
    a = [str(x) for x in a]
    df_corr = df_corr.loc[a,a]
    
    #save df_corr
    filename = os.path.join(SJdir, 'PCA', 'Stats', 'Networks', 'zscore', 'PCA_'+param, 'networks', 'thresholded_correlations', '%s_%s_%s.csv' %(subj, task, pc))
    df_corr.to_csv(filename)
    
    #plot
    f, ax = plt.subplots()

    df_thresh[idx][['elec', 'pattern', 'ROI']]
    b = zip(df_thresh[idx].elec, df_thresh[idx].pattern, df_thresh[idx].ROI)
    b = ['%i - %s - %s' %(x) for x in b]

    plt.imshow(df_corr, vmin = -1, vmax = 1, interpolation = 'none', cmap = plt.cm.RdBu_r)
    ax.set_xticks(range(len(a)))
    ax.set_yticks(range(len(a)))
    ax.set_xticklabels(a)
    ax.set_yticklabels(b)
    plt.colorbar()
    ax.set_title('%s %s - %s - %s' %(subj, task, param, pc))
    
    filename = os.path.join(SJdir, 'PCA', 'Stats', 'Networks', 'zscore', 'PCA_'+param, 'networks', 'thresholded_correlations', '%s_%s_%s.png' %(subj, task, pc))
    plt.savefig(filename)
    plt.close()

IOError: File /home/knight/matar/MATLAB/DATA/Avgusta/PCA/Stats/Networks/zscore/PCA_means/networks/CP7_DecisionAud_thresh.csv does not exist

## update PC csv with revised electrode designations

In [4]:
df = pd.read_csv(os.path.join(SJdir, 'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv'))
df_pca = pd.read_csv(os.path.join(SJdir, 'PCA', 'Stats', 'Networks', 'unsmoothed', 'PCA_medians', 'networks', 'all_subjects_thresh_medians_withclusters.csv'))

In [12]:
df_pca_new = pd.merge(df[['subj','task','elec','pattern']], df_pca[['subj','task','elec','cluster','max_pc','pc_list','ROI']])

In [25]:
filename = os.path.join(SJdir,'PCA', 'csvs_FINAL', 'all_subjects_thresh_medians_withclusters_newdesignations.csv')
df_pca_new.to_csv(filename)

## calculate proportion explained

In [4]:
filename = os.path.join(SJdir,'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df = pd.read_csv(os.path.join(filename))

subj_task = df[['subj', 'task']].drop_duplicates()

pc1 = []

for s_t in subj_task.itertuples():
    _, subj, task = s_t

    filename = os.path.join(SJdir, 'PCA','Stats', 'Networks','zscore','PCA_means','%s_%s_summary.txt' %(subj, task))
    list_of_lists = []
    with open(filename,'r') as f:
        for line in f:
            inner_list = [elt.strip() for elt in line.split(',')]
            # in alternative, if you need to use the file content as numbers
            # inner_list = [int(elt.strip()) for elt in line.split(',')]
            list_of_lists.append(inner_list)

        tmp = [x[0].split(' ') for x in list_of_lists if x[0].split(' ')[0:2] == ['Proportion', 'Var']][0]
        tmp = [t for t in tmp if len(t)>0]
        pc1.append(np.float(tmp[2]))

In [5]:
np.mean(pc1)

0.2481081081081081

In [7]:
stats.sem(pc1)

0.018469347375263259